# Camera Identification

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import *
from keras.layers import *
from keras import backend as K
from keras.utils.vis_utils import plot_model
import pydot
import graphviz
import PIL
import cv2
import glob
import os

Using TensorFlow backend.


Read in data

In [2]:
def read(img):
    return np.asfarray(PIL.Image.open(img))

In [3]:
input_path = 'data'
train_path = input_path + '/' + 'train' + '/'
test_path = input_path + '/' + 'test' + '/'
labels = os.listdir(train_path)

In [4]:
print labels

['Motorola-Nexus-6', 'Sony-NEX-7', 'Motorola-X', 'HTC-1-M7', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 'iPhone-4s', 'Samsung-Galaxy-S4', 'iPhone-6', 'Samsung-Galaxy-Note3']


In [33]:

def gettraining(labels,path):
    trainlabels = []
    for l in labels:
        for i in os.listdir(path + l + '/'):
            trainlabels.append((path + l + '/'+ str(i),l))

    train = pd.DataFrame(trainlabels, columns=['camera', 'name'])
    return train

def gettest(path):
    test = []
    ids = []
    for i in os.listdir(path):
        test.append(path + i)
        ids.append(i)
    return test, ids
train = gettraining(labels,train_path)
test, ids = gettest(test_path)

In [6]:
train.head(5)

,camera,name
0,data/train/Motorola-Nexus-6/(MotoNex6)40.jpg,Motorola-Nexus-6
1,data/train/Motorola-Nexus-6/(MotoNex6)143.jpg,Motorola-Nexus-6
2,data/train/Motorola-Nexus-6/(MotoNex6)202.jpg,Motorola-Nexus-6
3,data/train/Motorola-Nexus-6/(MotoNex6)80.jpg,Motorola-Nexus-6
4,data/train/Motorola-Nexus-6/(MotoNex6)39.jpg,Motorola-Nexus-6


In [7]:
test[0:5]

['data/test/img_00d0e3b_unalt.tif',
 'data/test/img_c6839e4_manip.tif',
 'data/test/img_b25b81e_unalt.tif',
 'data/test/img_c944786_manip.tif',
 'data/test/img_2cc6262_unalt.tif']

In [8]:
image = train["camera"][0]
img = cv2.imread(str(image),0)

In [9]:
print img

[[162 161 162 ...,  63  70  69]
 [162 162 162 ...,  71  76  75]
 [163 162 162 ...,  82  84  85]
 ..., 
 [ 44  44  44 ...,   6   6   5]
 [ 43  44  45 ...,   6   6   5]
 [ 41  43  45 ...,   5   5   5]]


In [10]:
img.shape

(3120, 4160)

In [11]:
resized_image = np.asfarray(cv2.resize(img, (32, 32)))
#cv2.imshow('image',img)

Lets flatten the image and divide by the max value(255) and flatten it into a 1d vector, so we can input it into a model

In [12]:
resized = resized_image.flatten()/255.0

In [13]:
print resized.shape

(1024,)

Do the rest for the other pictures

In [14]:
Xtrain = train["camera"]
y = train["name"]


In [15]:
def preprocess(path):
    img = cv2.imread(path,0)
    img = np.asfarray(cv2.resize(img,(32, 32)))
    return img.flatten()/255.0

def processbulk(paths):
    return np.asfarray([preprocess(filepath) for filepath in paths])

X_train = processbulk(Xtrain[0:5])

def label_transform(labels):
    labels = pd.get_dummies(pd.Series(labels))
    label_index = labels.columns.values
    
    return labels, label_index

In [16]:
X_train = processbulk(Xtrain)
X_test = processbulk(test)

In [17]:
y = label_transform(y)

In [18]:
y = np.array(y[0])

In [40]:
print y

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


## Multi Channel Convolutional Neural Network
Because the images probably have minor details, we want to use a convolutional neural network with multiple kernal sizes to account for local dependencies in the images.

Here we will input the flattened version of grayscale images into a multichannel convolutional neural network, with kernal size of 12,24, and 36 and 32 filters each.

In [39]:
class MultiChannelCNN:
    
    saved = 0
    
    def __init__(self, X,y,models=[]):
        self.X = np.expand_dims(X, axis=2) #need to add an extra column, 1d convolution needs to "slide" accross
        self.y = y
        self.models = models
        
    #channels is an integer, number of channels
    #kernel size is a list of dimensions for the kernels
    def createmodel(self,channels,kernel_size,num_filters):
        
        K.clear_session()

        inputlayers = {}
        layers = {}
        flats = {}
        length = self.X.shape[1]
        for i in range(channels):
            print i
            inputlayers["input"+ str(i)] = Input(shape=(length,1))
            print inputlayers["input"+str(i)]
            layers["conv" + str(i)] = Conv1D(filters=num_filters,input_shape=(length, 1), kernel_size=kernel_size[i], activation='relu')(inputlayers["input" + str(i)])
            layers["dropout" + str(i)] =  Dropout(0.5)(layers["conv" + str(i)])
            layers["pool" + str(i)] = MaxPooling1D(pool_size=4)(layers["dropout" + str(i)])
            flats["flat" + str(i)] = Flatten()(layers["pool" + str(i)])
        
        merge = concatenate(list(flats.values()))
        dense = Dense(10, activation='relu')(merge)
        outputs = Dense(10, activation='sigmoid')(dense)
        model = Model(inputs=list(inputlayers.values()), outputs=outputs)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        print(model.summary())
        self.models.append(model)
    
    #train using custom params
    def train(self,model,epochs,channels,batch_size):
        
        inp = []
        for i in range(channels):
            inp.append(self.X)
        
#         model.fit(inp, self.y,validation_split=0.1, epochs=epochs, batch_size=batch_size,verbose=1)
        model.fit(inp, self.y,validation_split=0.1, epochs=epochs,verbose=1)

        
        if MultiChannelCNN.saved < 1:
            model.save('multichannelcnn.h5')
        else:
            print("Already Saved")
        loss, acc = model.evaluate([self.X,self.X,self.X], self.y, verbose=0)
        print('Train Accuracy: %f' % (acc*100))
        
        return model
    
    #predict
    def predict(self,model,data):
        
        #model = load_model('multichannelcnn.h5')

        predicts = model.predict(data)

        return predicts 
        

In [24]:
mcnn = MultiChannelCNN(X_train,y)
mcnn.createmodel(3,[12,24,36],32)

0
Tensor("input_1:0", shape=(?, 1024, 1), dtype=float32)
1
Tensor("input_2:0", shape=(?, 1024, 1), dtype=float32)
2
Tensor("input_3:0", shape=(?, 1024, 1), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None

In [25]:
mcnn.models[0] = mcnn.train(mcnn.models[0],3,3,1)

Train on 2475 samples, validate on 275 samples
Epoch 1/3
2475/2475 [==============================] - 19s 8ms/step - loss: 2.2710 - acc: 0.1127 - val_loss: 2.9716 - val_acc: 0.0000e+00
Epoch 2/3
2475/2475 [==============================] - 20s 8ms/step - loss: 2.2205 - acc: 0.1390 - val_loss: 2.9755 - val_acc: 0.0000e+00
Epoch 3/3
2475/2475 [==============================] - 19s 8ms/step - loss: 2.1824 - acc: 0.1499 - val_loss: 3.6587 - val_acc: 0.0000e+00
Train Accuracy: 13.781818


In [26]:
print mcnn.models

In [27]:
newtest = np.expand_dims(X_test, axis=2)
out = mcnn.predict(mcnn.models[0],[newtest,newtest,newtest])

In [28]:
print X_test

array([[ 0.63529412,  0.67843137,  0.99607843, ...,  0.56862745,
         0.53333333,  0.56470588],
       [ 0.41176471,  0.2745098 ,  0.22745098, ...,  0.36078431,
         0.34901961,  0.30196078],
       [ 0.25490196,  0.20392157,  0.19215686, ...,  0.09411765,
         0.07843137,  0.08235294],
       ..., 
       [ 0.94509804,  0.94509804,  0.94509804, ...,  0.63137255,
         0.61568627,  0.61176471],
       [ 0.83137255,  0.83137255,  0.8       , ...,  0.34117647,
         0.34901961,  0.48235294],
       [ 0.15294118,  0.15686275,  0.14901961, ...,  0.20392157,
         0.18039216,  0.18039216]])

In [29]:
out

array([[ 0.3888211 ,  0.19045559,  0.20177774, ...,  0.36421365,
         0.31689966,  0.61973965],
       [ 0.47084528,  0.38578287,  0.37158355, ...,  0.45565501,
         0.43095353,  0.54299384],
       [ 0.51545525,  0.5219835 ,  0.50094211, ...,  0.51122487,
         0.5077641 ,  0.49633321],
       ..., 
       [ 0.44389102,  0.27445266,  0.21768001, ...,  0.39397815,
         0.32377276,  0.59678173],
       [ 0.49460456,  0.40248632,  0.30740657, ...,  0.44680113,
         0.38470557,  0.55284375],
       [ 0.360324  ,  0.26334158,  0.56876397, ...,  0.45948571,
         0.5700655 ,  0.53059053]], dtype=float32)

In [30]:
out
predict = np.argmax(out, axis=1)
predict = [labels[p] for p in predict]

In [31]:
print predict

['HTC-1-M7', 'HTC-1-M7', 'Sony-NEX-7', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'Motorola-X', 'HTC-1-M7', 'HTC-1-M7', 'Samsung-Galaxy-Note3', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'iPhone-4s', 'LG-Nexus-5x', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'Motorola-Nexus-6', 'HTC-1-M7', 'Motorola-X', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'Motorola-X', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'Motorola-X', 'HTC-1-M7', 'HTC-1-M7', 'Motorola-X', 'HTC-1-M7', 'Motorola-X', 'HTC-1-M7', 'LG-Nexus-5x', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'Motorola-X', 'iPhone-4s', 'LG-Nexus-5x', 'HTC-1-M7', 'HTC-1-M7', 'HTC-1-M7', 'iPhone-4s', 'HTC-1-M7', 'HTC-1-M7', 'H

Submit to csv

In [35]:
df = pd.DataFrame(columns=['fname', 'camera'])
df["fname"] = ids
df["camera"] = predict


In [36]:
df

,fname,camera
0,img_00d0e3b_unalt.tif,HTC-1-M7
1,img_c6839e4_manip.tif,HTC-1-M7
2,img_b25b81e_unalt.tif,Sony-NEX-7
3,img_c944786_manip.tif,HTC-1-M7
4,img_2cc6262_unalt.tif,iPhone-4s
5,img_31ec479_manip.tif,HTC-1-M7
6,img_2afa619_unalt.tif,iPhone-4s
7,img_60fa981_manip.tif,HTC-1-M7
8,img_dce2359_manip.tif,Motorola-X
9,img_ad161ee_unalt.tif,HTC-1-M7


In [38]:
df.to_csv("data/submissions/3channelconvnet.csv", index=False)